In [4]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [5]:
from azureml.core import ComputeTarget

# name for your cluster
cpu_cluster_name = "general"

# check if cluster already exists
cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
print('Found existing cluster, use it.')

Found existing cluster, use it.


In [6]:
from azureml.core import Environment

#env = Environment('lss')
#env.docker.base_dockerfile = './Dockerfile' # path to your dockerfile
env = Environment.get(ws, 'lss', version=1)

In [7]:
from azureml.core import Dataset

ds = ws.datastores['mini']
mini = Dataset.File.from_files(path=(ds, '/'))

ds = ws.datastores['maps']
maps = Dataset.File.from_files(path=(ds, '/'))

In [9]:
from azureml.data import OutputFileDatasetConfig

def_blob_store = ws.get_default_datastore()
output = OutputFileDatasetConfig(destination=(def_blob_store, 'sample/outputmodel'))

In [17]:
from azureml.core import ScriptRunConfig

from azureml.core import Experiment

#arg = "python main.py multi_train trainval --logdir=./run --gpuid=-1".split().extend([ "--dataroot="+mini.as_mount(), "--map_folder="+maps.as_mount() ])
arguments=[mini.as_mount('mini'), maps.as_mount('maps'), output]
print(arguments)
config = ScriptRunConfig(source_directory='..',
                            script='train_az.py',
                            compute_target=cpu_cluster,
                            environment=env,
                            arguments=arguments
                            )

exp = Experiment(ws, 'test2')
run = exp.submit(config)
print(run)
run.wait_for_completion(show_output=True)

[<azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x00000275C2D88AC8>, <azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x00000275C2D92088>, <azureml.data.output_dataset_config.OutputFileDatasetConfig object at 0x00000275C06216C8>]
Run(Experiment: test2,
Id: test2_1633928043_9fbffbb5,
Type: azureml.scriptrun,
Status: Preparing)
RunId: test2_1633928043_9fbffbb5
Web View: https://ml.azure.com/runs/test2_1633928043_9fbffbb5?wsid=/subscriptions/aab48da9-d313-43b6-a34e-5f6def1a9246/resourcegroups/PilotierA/workspaces/PilotierA&tid=7a5d1346-2f6f-43ed-a996-c114a2c559a5

Execution Summary
RunId: test2_1633928043_9fbffbb5
Web View: https://ml.azure.com/runs/test2_1633928043_9fbffbb5?wsid=/subscriptions/aab48da9-d313-43b6-a34e-5f6def1a9246/resourcegroups/PilotierA/workspaces/PilotierA&tid=7a5d1346-2f6f-43ed-a996-c114a2c559a5

Warnings:
This run is using a new job runtime with improved performance and error reporting. The logs from your script

{'runId': 'test2_1633928043_9fbffbb5',
 'target': 'general',
 'status': 'Completed',
 'startTimeUtc': '2021-10-11T04:54:06.212943Z',
 'endTimeUtc': '2021-10-11T04:55:47.974825Z',
 'services': {},
 'warnings': [{'message': 'This run is using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}],
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '05cd4861-6e14-4d9e-b066-b9c075cc4ff3',
  'azureml.git.repository_uri': 'https://github.com/pilotier/lift-splat-shoot.git',
  'mlflow.source.git.repoURL': 'https://github.com/pilotier/lift-splat-shoot.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '25366

In [ ]:
run = exp.submit(config)
print(run)
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.tensorboard import Tensorboard

tb = Tensorboard([run])

# If successful, start() returns a string with the URI of the instance.
tb.start()

In [ ]:
# After your job completes, be sure to stop() the streaming otherwise it will continue to run. 
tb.stop()

## Other tools

In [18]:
from src.tools import generate_video_from_imgs

generate_video_from_imgs("/output", ".jpg")

ModuleNotFoundError: No module named 'src'

In [15]:
import os
os.listdir('../src')

['data.py',
 'explore.py',
 'models.py',
 'tools.py',
 'train.py',
 '__init__.py',
 '__pycache__']